In [1]:
import numpy as np

# One-hot encoding
In the beginning were the words. So very many words. Our first step is to convert all the words to numbers so we can do math on them.

Imagine that our goal is to create the computer that responds to our voice commands. It’s our job to build the transformer that converts (or transduces) a sequence of sounds to a sequence of words.

We start by choosing our vocabulary, the collection of symbols that we are going to be working with in each sequence. In our case, there will be two different sets of symbols, one for the input sequence to represent vocal sounds and one for the output sequence to represent words.

For now, let's assume we're working with English. There are tens of thousands of words in the English language, and perhaps another few thousand to cover computer-specific terminology. That would give us a vocabulary size that is the better part of a hundred thousand. One way to convert words to numbers is to start counting at one and assign each word its own number. Then a sequence of words can be represented as a list of numbers.

For example, consider a tiny language with a vocabulary size of three: files, find, and my. Each word could be swapped out for a number, perhaps files = 1, find = 2, and my = 3. Then the sentence "Find my files", consisting of the word sequence [ find, my, files ] could be represented instead as the sequence of numbers [2, 3, 1].

This is a perfectly valid way to convert symbols to numbers, but it turns out that there's another format that's even easier for computers to work with, one-hot encoding. In one-hot encoding a symbol is represented by an array of mostly zeros, the same length of the vocabulary, with only a single element having a value of one. Each element in the array corresponds to a separate symbol.

Another way to think about one-hot encoding is that each word still gets assigned its own number, but now that number is an index to an array. Here is our example above, in one-hot notation.

In [2]:
entire_corpus = "Find my files".lower()
corpus = sorted(entire_corpus.split())
print(corpus)

['files', 'find', 'my']


![](https://e2eml.school/images/transformers/one_hot_vocabulary.png)

So the sentence "Find my files" becomes a sequence of one-dimensional arrays, which, after you squeeze them together, starts to look like a two-dimensional array.

In [3]:
# create function to return one-hot vector from a list of words
def one_hot_vector(word_list, corpus):
    one_hot_vector = []
    for word in word_list:
        one_hot_vector.append([1 if word == corpus[i] else 0 for i in range(len(corpus))])
    return one_hot_vector
print(corpus)
one_hot_vector(corpus, corpus)

['files', 'find', 'my']


[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

![](https://e2eml.school/images/transformers/one_hot_sentence.png)

Heads-up, I'll be using the terms "one-dimensional array" and "vector" interchangeably. Likewise with "two-dimensional array" and "matrix".


In [4]:
def word_to_one_hot(word, corpus):
    return one_hot_vector([word], corpus)[0]

[word_to_one_hot(word, corpus) for word in ["find", "my", "files"]]

[[0, 1, 0], [0, 0, 1], [1, 0, 0]]

# Dot product
One really useful thing about the one-hot representation is that it lets us compute dot products. These are also known by other intimidating names like inner product and scalar product. To get the dot product of two vectors, multiply their corresponding elements, then add the results.

![](https://e2eml.school/images/transformers/dot_product.png)

In [5]:
array_1 = np.array([0, 1, 1, 2])
array_2 = np.array([1, 0, 1, 2])
array_1.dot(array_2)

5

Dot products are especially useful when we're working with our one-hot word representations. The dot product of any one-hot vector with itself is one.

![](https://e2eml.school/images/transformers/match.png)

In [6]:
array_1 = np.array([0, 1, 0, 0])
array_2 = np.array([0, 1, 0, 0])
array_1.dot(array_2)

1

And the dot product of any one-hot vector with any other one-hot vector is zero.

![](https://e2eml.school/images/transformers/non_match.png)

In [7]:
array_1 = np.array([0, 1, 0, 0])
array_2 = np.array([0, 0, 0, 1])
array_1.dot(array_2)

0

The previous two examples show how dot products can be used to measure similarity. As another example, consider a vector of values that represents a combination of words with varying weights. A one-hot encoded word can be compared against it with the dot product to show how strongly that word is represented.

![](https://e2eml.school/images/transformers/similarity.png)

In [8]:
array_1 = np.array([0, 0, 1, 0]) # Word
array_2 = np.array([0.2, 0.7, 0.8, 0.1]) # Combination of words
array_1.dot(array_2)

0.8

# Matrix Multiplication

The dot product is the building block of matrix multiplication, a very particular way to combine a pair of two-dimensional arrays. We'll call the first of these matrices A and the second one B. In the simplest case, when A has only one row and B has only one column, the result of matrix multiplication is the dot product of the two.

![](https://e2eml.school/images/transformers/matrix_mult_one_row_one_col.png)

Notice how the number of columns in A and the number of rows in B needs to be the same for the two arrays to match up and for the dot product to work out.

In [9]:
# matrix multiply the vectors [0, 0, 1, 0] and [0.2, 0.7, 0.8, 0.1]
np.matmul(np.array([0, 0, 1, 0]), np.array([0.2, 0.7, 0.8, 0.1]))

0.8

When A and B start to grow, matrix multiplication starts to get trippy. To handle more than one row in A, take the dot product of B with each row separately. The answer will have as many rows as A does.

![](https://e2eml.school/images/transformers/matrix_mult_two_row_one_col.png)

In [10]:
np.matmul(np.array([[1, 0, 0, 0], [0, 0, 1, 0]]), np.array([0.2, 0.7, 0.8, 0.1]))

array([0.2, 0.8])

When B takes on more columns, take the dot product of each column with A and stack the results in successive columns.

![](https://e2eml.school/images/transformers/matrix_mult_one_row_two_col.png)

In [11]:
# matrix multiply the vectors [0, 0, 1, 0] and [[0.2, 0.7, 0.8, 0.1], [0.9, 0, 0.3, 0.4]]
np.dot(np.array([0, 0, 1, 0]), np.array([[0.2, 0.7, 0.8, 0.1], [0.9, 0, 0.3, 0.4]]).transpose())

array([0.8, 0.3])

Now we can extend this to mutliplying any two matrices, as long as the number of columns in A is the same as the number of rows in B. The result will have the same number of rows as A and the same number of columns as B.

![](https://e2eml.school/images/transformers/matrix_mult_three_row_two_col.png)

In [12]:
a = np.array([
    [1, 0, 0, 0], 
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])
b = np.array([
    [0.2, 0.7, 0.8, 0.1], 
    [0.9, 0, 0.3, 0.4]
])
np.matmul(a, b.transpose())

array([[0.2, 0.9],
       [0.1, 0.4],
       [0.8, 0.3]])

If this is the first time you're seeing this, it might feel needlessly complex, but I promise it pays off later.

## Matrix multiplication as a table lookup
Notice how matrix multiplication acts as a lookup table here. Our A matrix is made up of a stack of one-hot vectors. They have ones in the first column, the fourth column, and the third column, respectively. When we work through the matrix multiplication, this serves to pull out the first row, the fourth row, and the third row of the B matrix, in that order. This trick of using a one-hot vector to pull out a particular row of a matrix is at the core of how transformers work.

# First order sequence model
We can set aside matrices for a minute and get back to what we really care about, sequences of words. Imagine that as we start to develop our natural language computer interface we want to handle just three possible commands:

```
Show me my directories please.
Show me my files please.
Show me my photos please.
```
Our vocabulary size is now seven:
`{directories, files, me, my, photos, please, show}`.

One useful way to represent sequences is with a transition model. For every word in the vocabulary, it shows what the next word is likely to be. If users ask about photos half the time, files 30% of the time, and directories the rest of the time, the transition model will look like this. The sum of the transitions away from any word will always add up to one.

![](https://e2eml.school/images/transformers/markov_chain.png)

This particular transition model is called a **Markov chain**, because it satisfies the [Markov property](https://en.wikipedia.org/wiki/Markov_property) that the probabilities for the next word depend only on recent words. More specifically, it is a first order Markov model because it only looks at the single most recent word. If it considered the two most recent words it would be a second order Markov model.

In [13]:
import pandas as pd
weighted_edge_list = [
    ('show', 'me', 1), 
    ('me', 'my', 1), 
    ('my', 'directories', 0.2),
    ('my', 'files', 0.3),
    ('my', 'photos', 0.5),
    ('directories', 'please', 1),
    ('files', 'please', 1),
    ('photos', 'please', 1),
    ('please', 'please', 0)
]
wel_df = pd.DataFrame(weighted_edge_list, columns=["source", "target", "weight"])
wel_df

,source,target,weight
0,show,me,1.0
1,me,my,1.0
2,my,directories,0.2
3,my,files,0.3
4,my,photos,0.5
5,directories,please,1.0
6,files,please,1.0
7,photos,please,1.0
8,please,please,0.0


Our break from matrices is over. It turns out that Markov chains can be expressed conveniently in matrix form. Using the same indexing scheme that we used when creating one-hot vectors, each row represents one of the words in our vocabulary. So does each column. The matrix transition model treats a matrix as a lookup table. Find the row that corresponds to the word you’re interested in. The value in each column shows the probability of that word coming next. Because the value of each element in the matrix represents a probability, they will all fall between zero and one. Because probabilities always sum to one, the values in each row will always add up to one.

![](https://e2eml.school/images/transformers/transition_matrix.png)

In the transition matrix here we can see the structure of our three sentences clearly. Almost all of the transition probabilities are zero or one. There is only one place in the Markov chain where branching happens. After my, the words directories, files, or photos might appear, each with a different probability. Other than that, there’s no uncertainty about which word will come next. That certainty is reflected by having mostly ones and zeros in the transition matrix.

In [14]:
transition_matrix_df = wel_df.pivot(index="source", columns="target", values="weight").fillna(0)
transition_matrix_df

target,directories,files,me,my,photos,please
source,,,,,,
directories,0.0,0.0,0.0,0.0,0.0,1.0
files,0.0,0.0,0.0,0.0,0.0,1.0
me,0.0,0.0,0.0,1.0,0.0,0.0
my,0.2,0.3,0.0,0.0,0.5,0.0
photos,0.0,0.0,0.0,0.0,0.0,1.0
please,0.0,0.0,0.0,0.0,0.0,0.0
show,0.0,0.0,1.0,0.0,0.0,0.0


We can revisit our trick of using matrix multiplication with a one-hot vector to pull out the transition probabilities associated with any given word. For instance, if we just wanted to isolate the probabilities of which word comes after my, we can create a one-hot vector representing the word my and multiply it by our transition matrix. This pulls out the relevant row and shows us the probability distribution of what the next word will be.

![](https://e2eml.school/images/transformers/transition_lookups.png)

In [15]:
transition_matrix = transition_matrix_df.to_numpy()
transition_matrix

array([[0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 1. , 0. , 0. ],
       [0.2, 0.3, 0. , 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ]])

In [16]:
word_vector = np.array([0, 0, 0, 1, 0, 0, 0])

word_vector.dot(transition_matrix)

array([0.2, 0.3, 0. , 0. , 0.5, 0. ])

# Second order sequence model
Predicting the next word based on only the current word is hard. That's like predicting the rest of a tune after being given just the first note. Our chances are a lot better if we can at least get two notes to go on.

We can see how this works in another toy language model for our computer commands. We expect that this one will only ever see two sentences, in a 40/60 proportion.

```
Check whether the battery ran down please.
Check whether the program ran please.
```
A Markov chain illustrates a first order model for this.

![](https://e2eml.school/images/transformers/markov_chain_2.png)

In [17]:
def build_markov_chain(sentences_with_weights):
    import re
    from collections import defaultdict

    # Initialize a dictionary to store transition counts
    transition_counts = defaultdict(lambda: defaultdict(int))

    # Process each sentence
    for sentence, weight in sentences_with_weights:
        words = re.findall(r'\b\w+\b', sentence.lower())  # Tokenize the sentence
        for i in range(len(words) - 1):
            transition_counts[words[i]][words[i+1]] += weight  # Update counts

    # Convert counts to probabilities
    markov_chain = {}
    for current_word, transitions in transition_counts.items():
        total = sum(transitions.values())
        markov_chain[current_word] = {word: count / total for word, count in transitions.items()}

    return markov_chain

# Example usage
sentences_with_weights = [
    ("Check whether the battery ran down please.", 0.4),
    ("Check whether the program ran please.", 0.6)
]
markov_chain = build_markov_chain(sentences_with_weights)
markov_chain


{'check': {'whether': 1.0},
 'whether': {'the': 1.0},
 'the': {'battery': 0.4, 'program': 0.6},
 'battery': {'ran': 1.0},
 'ran': {'down': 0.4, 'please': 0.6},
 'down': {'please': 1.0},
 'program': {'ran': 1.0}}

Here we can see that if our model looked at the two most recent words, instead of just one, that it could do a better job. When it encounters battery ran, it knows that the next word will be down, and when it sees program ran the next word will be please. This eliminates one of the branches in the model, reducing uncertainty and increasing confidence. Looking back two words turns this into a second order Markov model. It gives more context on which to base next word predictions. Second order Markov chains are more challenging to draw, but here are the connections that demonstrate their value.

![](https://e2eml.school/images/transformers/markov_chain_second_order.png)

To highlight the difference between the two, here is the first order transition matrix,

![](https://e2eml.school/images/transformers/transition_matrix_first_order_2.png)

In [18]:
def create_transition_matrix_df(markov_chain):
    # Identify all unique states
    states = set(markov_chain.keys())
    for transitions in markov_chain.values():
        states.update(transitions.keys())

    # Convert states set to a list to maintain order
    states = list(states)

    # Create state index mapping
    state_index = {state: i for i, state in enumerate(states)}

    # Initialize the transition matrix
    n = len(states)
    transition_matrix = np.zeros((n, n))

    # Populate the matrix
    for state, transitions in markov_chain.items():
        state_idx = state_index[state]
        for next_state, prob in transitions.items():
            next_state_idx = state_index[next_state]
            transition_matrix[state_idx][next_state_idx] = prob

    # Convert to DataFrame
    matrix_df = pd.DataFrame(transition_matrix, index=states, columns=states)
    return matrix_df

transition_matrix = create_transition_matrix_df(markov_chain)
transition_matrix


,please,down,ran,check,battery,the,whether,program
please,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
down,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ran,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.0
check,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
battery,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
the,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.6
whether,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
program,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


and here is the second order transition matrix.

![](https://e2eml.school/images/transformers/transition_matrix_second_order.png)

In [19]:
from collections import defaultdict

def create_weighted_transition_dataframe(sentences, weights):
    # Ensure weights sum up to 1
    total_weight = sum(weights)
    normalized_weights = [w / total_weight for w in weights]

    # Tokenize the sentences and create transitions with weights
    transitions = defaultdict(lambda: defaultdict(float))
    for sentence, weight in zip(sentences, normalized_weights):
        words = sentence.lower().split()
        for i in range(len(words) - 2):
            current_pair = (words[i], words[i + 1])
            next_word = words[i + 2]
            transitions[current_pair][next_word] += weight

    # Create a DataFrame from the transitions
    df = pd.DataFrame.from_dict({k: dict(v) for k, v in transitions.items()}, orient='index').fillna(0)

    # Normalize to get probabilities
    df = df.div(df.sum(axis=1), axis=0)

    return df

# Define the sentences and their weights
sentences = [
    "Check whether the battery ran down please",
    "Check whether the program ran please"
]
weights = [0.4, 0.6]  # 40% for the first sentence, 60% for the second

# Generate the weighted transition DataFrame
weighted_transition_df = create_weighted_transition_dataframe(sentences, weights)

# Print the DataFrame
weighted_transition_df


the  battery  program  ran  down  please
check   whether  1.0      0.0      0.0  0.0   0.0     0.0
whether the      0.0      0.4      0.6  0.0   0.0     0.0
the     battery  0.0      0.0      0.0  1.0   0.0     0.0
        program  0.0      0.0      0.0  1.0   0.0     0.0
battery ran      0.0      0.0      0.0  0.0   1.0     0.0
ran     down     0.0      0.0      0.0  0.0   0.0     1.0
program ran      0.0      0.0      0.0  0.0   0.0     1.0

Notice how the second order matrix has a separate row for every combination of words (most of which are not shown here). That means that if we start with a vocabulary size of N then the transition matrix has N^2 rows.

What this buys us is more confidence. There are more ones and fewer fractions in the second order model. There's only one row with fractions in it, one branch in our model. Intuitively, looking at two words instead of just one gives more context, more information on which to base a next word guess.

# Second order sequence model with skips

A second order model works well when we only have to look back two words to decide what word comes next. What about when we have to look back further? Imagine we are building yet another language model. This one only has to represent two sentences, each equally likely to occur.
```
Check the program log and find out whether it ran please.
Check the battery log and find out whether it ran down please.
```
In this example, in order to determine which word should come after ran, we would have to look back 8 words into the past. If we want to improve on our second order language model, we can of course consider third- and higher order models. However, with a significant vocabulary size this takes a combination of creativity and brute force to execute. A naive implementation of an eighth order model would have N^8 rows, a ridiculous number for any reasonable vocubulary.

Instead, we can do something sly and make a second order model, but consider the combinations of the most recent word with each of the words that came before. It's still second order, because we're only considering two words at a time, but it allows us to reach back further and capture **long range dependencies**. The difference between this second-order-with-skips and a full umpteenth-order model is that we discard most of the word order information and combinations of preceeeding words. What remains is still pretty powerful.

Markov chains fail us entirely now, but we can still represent the link between each pair of preceding words and the words that follow. Here we've dispensed with numerical weights, and instead are showing only the arrows associated with non-zero weights. Larger weights are shown with heavier lines.

![](https://e2eml.school/images/transformers/feature_voting.png)

Here's what it might look like in a transition matrix.


![](https://e2eml.school/images/transformers/transition_matrix_second_order_skips.png)

This view only shows the rows relevant to predicting the word that comes after ran. It shows instances where the most recent word (ran) is preceded by each of the other words in the vocabulary. Only the relevant values are shown. All the empty cells are zeros.

The first thing that becomes apparent is that, when trying to predict the word that comes after ran, we no longer look at just one line, but rather a whole set of them. We've moved out of the Markov realm now. Each row no longer represents the state of the sequence at a particular point. Instead, each row represents one of many **features** that may describe the sequence at a particular point. The combination of the most recent word with each of the words that came before makes for a collection of applicable rows, maybe a large collection. Because of this change in meaning, each value in the matrix no longer represents a probability, but rather a vote. Votes will be summed and compared to determine next word predictions.

The next thing that becomes apparent is that most of the features don't matter. Most of the words appear in both sentences, and so the fact that they have been seen is of no help in predicting what comes next. They all have a value of .5. The only two exceptions are battery and program. They have some 1 and 0 weights associated with the two cases we're trying to distinguish. The feature battery, ran indicates that ran was the most recent word and that battery occurred somewhere earlier in the sentence. This feature has a weight of 1 associated with down and a weight of 0 associated with please. Similarly, the feature program, ran has the opposite set of weights. This structure shows that it is the presence of these two words earlier in the sentence that is decisive in predicting which word comes next.

To convert this set of word-pair features into a next word estimate, the values of all the relevant rows need to be summed. Adding down the column, the sequence Check the program log and find out whether it ran generates sums of 0 for all the words, except a 4 for down and a 5 for please. The sequence Check the battery log and find out whether it ran does the same, except with a 5 for down and a 4 for please. By choosing the word with the highest vote total as the next word prediction, this model gets us the right answer, despite having an eight word deep dependency.

In [20]:
def create_transition_with_skips(sentences):
    # Tokenize the sentences and create pairs of words with and without skips
    transitions = defaultdict(lambda: defaultdict(int))
    for sentence in sentences:
        words = sentence.lower().split()
        for i in range(len(words) - 1):
            for skip in range(1, len(words) - i):
                current_pair = (words[i], words[i + skip])
                if i + skip + 1 < len(words):
                    next_word = words[i + skip + 1]
                    transitions[current_pair][next_word] += 1

    # Create a DataFrame from the transitions
    df = pd.DataFrame.from_dict({k: dict(v) for k, v in transitions.items()}, orient='index').fillna(0)

    # Normalize to get probabilities
    df = df.div(df.sum(axis=1), axis=0)

    return df

# Define the sentences
sentences = [
    "Check the program log and find out whether it ran please",
    "Check the battery log and find out whether it ran down please"
]

# Generate the transition DataFrame with skips
transition_df_with_skips = create_transition_with_skips(sentences)

# Columns like image
# Note 'check' and 'the' are not included because they have no transitions
cols = ['and', 'battery', 'down', 'find', 'it', 'log', 'out', 'please', 'program', 'ran', 'whether']
# subset the dataframe where the second element in the index is "ran"
ran_transitions = transition_df_with_skips.loc[transition_df_with_skips.index.get_level_values(1) == "ran"][cols]
ran_transitions


,,and,battery,down,find,it,log,out,please,program,ran,whether
check,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
the,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
program,ran,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
log,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
and,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
find,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
out,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
whether,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
it,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
battery,ran,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# filter out the index where the first element is in "Check the program log and find out whether it ran"
word_list_program = "Check the program log and find out whether it ran".lower().split()
word_list_battery = "Check the battery log and find out whether it ran".lower().split()
program_ran_transitions = ran_transitions.loc[ran_transitions.index.get_level_values(0).isin(word_list_program)][cols]
battery_ran_transitions = ran_transitions.loc[ran_transitions.index.get_level_values(0).isin(word_list_battery)][cols]

In [22]:
program_ran_transitions.sum(axis=0)

and        0.0
battery    0.0
down       4.0
find       0.0
it         0.0
log        0.0
out        0.0
please     5.0
program    0.0
ran        0.0
whether    0.0
dtype: float64

In [23]:
battery_ran_transitions.sum(axis=0)

and        0.0
battery    0.0
down       5.0
find       0.0
it         0.0
log        0.0
out        0.0
please     4.0
program    0.0
ran        0.0
whether    0.0
dtype: float64

# Masking
On more careful consideration, this is unsatisfying. The difference between a vote total of 4 and 5 is relatively small. It suggests that the model isn't as confident as it could be. And in a larger, more organic language model it's easy to imagine that such a slight difference could be lost in the statistical noise.

We can sharpen the prediction by weeding out all the uninformative feature votes. With the exception of battery, ran and program, ran. It's helpful to remember at this point that we pull the [relevant rows](https://e2eml.school/transformers.html#table_lookup) out of the transition matrix by multiplying it with a vector showing which features are currently active. For this example so far, we've been using the implied feature vector shown here.

![](https://e2eml.school/images/transformers/feature_selection.png)

In [24]:
implied_feature_vector = ((transition_df_with_skips.index.get_level_values(1) == "ran") & 
                        transition_df_with_skips.index.get_level_values(0).isin(word_list_battery))*1
implied_feature_vector

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [25]:
transition_df_with_skips.index[implied_feature_vector == 1]

MultiIndex([(  'check', 'ran'),
            (    'the', 'ran'),
            (    'log', 'ran'),
            (    'and', 'ran'),
            (   'find', 'ran'),
            (    'out', 'ran'),
            ('whether', 'ran'),
            (     'it', 'ran'),
            ('battery', 'ran')],
           )

It includes a one for each feature that is a combination of ran with each of the words that come before it. Any words that come after it don't get included in the feature set. (In the next word prediction problem these haven't been seen yet, and so it's not fair to use them predict what comes next.) And this doesn't include all the other possible word combinations. We can safely ignore these for this example because they will all be zero.

To improve our results, we can additionally force the unhelpful features to zero by creating a mask. It's a vector full of ones except for the positions you'd like to hide or mask, and those are set to zero. In our case we'd like to mask everything except for battery, ran and program, ran, the only two features that have been of any help.

![](https://e2eml.school/images/transformers/masked_feature_activities.png)

In [26]:
ran_transitions

,,and,battery,down,find,it,log,out,please,program,ran,whether
check,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
the,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
program,ran,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
log,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
and,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
find,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
out,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
whether,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
it,ran,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
battery,ran,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
feature_activities = (ran_transitions.index.get_level_values(0).isin(word_list_battery))*1
feature_activities

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [28]:
mask = (ran_transitions.index.get_level_values(0).isin(['battery', 'program']))*1
mask

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1])

In [29]:
masked_feature_activities = feature_activities * mask
masked_feature_activities

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [30]:
ran_transitions.loc[masked_feature_activities == 1] # next word is "down"

,,and,battery,down,find,it,log,out,please,program,ran,whether
battery,ran,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


To apply the mask, we multiply the two vectors element by element. Any feature activity value in an unmasked position will be multiplied by one and left unchanged. Any feature activity value in a masked position will be multiplied by zero, and thus forced to zero.

The mask has the effect of hiding a lot of the transition matrix. It hides the combination of ran with everything except battery and program, leaving just the features that matter.

![](https://e2eml.school/images/transformers/masked_transition_matrix.png)

In [31]:
ran_transitions[mask.astype(bool)]

,,and,battery,down,find,it,log,out,please,program,ran,whether
program,ran,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
battery,ran,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


After masking the unhelpful features, the next word predictions become much stronger. When the word battery occurs earlier in the sentence, the word after ran is predicted to be down with a weight of 1 and please with a weight of 0. What was a weight difference of 25 percent has become a difference of infinity percent. There is no doubt what word comes next. The same strong prediction occurs for please when program occurs early on.

This process of selective masking is the **attention** called out in the title of the original paper on transformers. So far, what we've descibed is a just an approximation of how attention is implemented in the paper. It captures the important concepts, but the details are different. We'll close that gap later.

# Attention as matrix multiplication
Feature weights could be straightforward to build by counting how often each word pair/next word transition occurs in training, but attention masks are not. Up to this point, we've pulled the mask vector out of thin air. How transformers find the relevant mask matters. It would be natural to use some sort of lookup table, but now we are focusing hard on expressing everything as matrix multiplications. We can use the same lookup method we introduced above by stacking the mask vectors for every word into a matrix and using the one-hot representation of the most recent word to pull out the relevant mask.

![](https://e2eml.school/images/transformers/mask_matrix_lookup.png)

In [34]:
# truncated example
Q = np.array([0, 1, 0])
KsuperT = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], # <-- target
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
])
np.matmul(Q, KsuperT)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In the matrix showing the collection of mask vectors, we've only shown the one we're trying to pull out, for clarity.

We're finally getting to the point where we can start tying into the paper. This mask lookup is represented by the QK^T term in the attention equation.

![Attention equation highlighting QKT](https://e2eml.school/images/transformers/attention_equation_QKT.png)

The query Q represents the feature of interest and the matrix K represents the collection of masks. Because it's stored with masks in columns, rather than rows, it needs to be transposed (with the T operator) before multiplying. By the time we're all done, we'll make some important modifications to this, but at this level it captures the concept of a differentiable lookup table that transformers make use of.

# Second order sequence model as matrix multiplications
Another step that we have been hand wavy about so far is the construction of transition matrices. We have been clear about the logic, but not about how to do it with matrix multiplications.

Once we have the result of our attention step, a vector that includes the most recent word and a small collection of the words that have preceded it, we need to translate that into features, each of which is a word pair. Attention masking gets us the raw material that we need, but it doesn’t build those word pair features. To do that, we can use a single layer fully connected neural network.

To see how a neural network layer can create these pairs, we'll hand craft one. It will be artificially clean and stylized, and its weights will bear no resemblance to the weights in practice, but it will demonstrate how the neural network has the expressivity necessary to build these two word pair features. To keep it small and clean, will focus on just the three attended words from this example, *battery, program, ran*.

![Neural network layer for creating multi word features](https://e2eml.school/images/transformers/feature_creation_layer.png)

In the layer diagram above, we can see how the weights act to combine the presence and absence of each word into a collection of features. This can also be expressed in matrix form.

![](https://e2eml.school/images/transformers/feature_creation_matrix.png)

In [39]:
edge_list = [
    ('battery', ('battery', 'run'), 1),    
    ('battery', ('program', 'run'), -1),
    ('program', ('battery', 'run'), -1),
    ('program', ('program', 'run'), 1),
    ('run', ('battery', 'run'), 1),
    ('run', ('program', 'run'), 1),
    ('bias', ('battery', 'run'), -1),
    ('bias', ('program', 'run'), -1),
]
el_df = pd.DataFrame(edge_list, columns=["source", "target", "weight"])
# pivot the dataframe
el_pivot = el_df.pivot(index="source", columns="target", values="weight").fillna(0).loc[['battery', 'program', 'run', 'bias']]
el_pivot

target,"(battery, run)","(program, run)"
source,,
battery,1,-1
program,-1,1
run,1,1
bias,-1,-1


In [47]:
# Using boolean indexing
seen_so_far = np.array([1, 0, 1, 1])
el_pivot[seen_so_far == 1].sum(axis=0)

target
(battery, run)    1
(program, run)   -1
dtype: int64

In [52]:
# Using matrix multiplication
el_matrix = el_pivot.to_numpy()
seen_so_far.dot(el_matrix) # first element = (battery, run); second element = (program, run)

array([ 1, -1], dtype=int64)

The battery and ran elements are 1 and the program element is 0. The bias element is always 1, a feature of neural networks. Working through the matrix multiplication gives a 1 for the element representing battery, ran and a -1 for the element representing program, ran. The results for the other case are similar.

![](https://e2eml.school/images/transformers/second_order_feature_program.png)

In [53]:
# Using boolean indexing
seen_so_far = np.array([0, 1, 1, 1])
el_pivot[seen_so_far == 1].sum(axis=0)

target
(battery, run)   -1
(program, run)    1
dtype: int64

In [54]:
# Using matrix multiplication
el_matrix = el_pivot.to_numpy()
seen_so_far.dot(el_matrix) # first element = (battery, run); second element = (program, run)

array([-1,  1], dtype=int64)

The final step in calculating these word combo features is to apply a rectified linear unit (ReLU) nonlinearity. The effect of this is to substitute any negative value with a zero. This cleans up both of these results so they represent the presence (with a 1) or absence (with a 0) of each word combination feature.

With those gymnastics behind us, we finally have a matrix multiplication based method for creating multiword features. Although I originally claimed that these consist of the most recent word and one earlier word, a closer look at this method shows that it can build other features too. When the feature creation matrix is learned, rather than hard coded, other structures can be learned. Even in this toy example, there's nothing to stop the creation of a three-word combination like battery, program, ran. If this combination occurred commonly enough it would probably end up being represented. There wouldn't be any way to indicated what order the words occurred in (at least not yet), but we could absolutely use their co-occurrence to make predictions. It would even be possible to make use of word combos that ignored the most recent word, like battery, program. These and other types of features are probably created in practice, exposing the oversimiplification I made when I claimed that transformers are a selective-second-order-with-skips sequence model. There's more nuance to it than that, and now you can see exactly what that nuance is. This won't be the last time we'll change the story to incorporate more subtlety.

In this form, the multiword feature matrix is ready for one more matrix multiplication, the second order sequence model with skips we developed above. All together, the sequence of

feature creation matrix multiplication,
ReLU nonlinearity, and
transition matrix multiplication

are the feedforward processing steps that get applied after attention is applied. Equation 2 from the paper shows these steps in a concise mathematical formulation.

![](https://e2eml.school/images/transformers/feedforward_equations.png)


*The Figure 1 architecture diagram of the of the paper shows these lumped together as the Feed Forward block.*

![](https://e2eml.school/images/transformers/architecture_feedforward.png)



# Sequence completion
So far we've only talked about next word prediction. There are a couple of pieces we need to add to get our decoder to generate a long sequence. The first is a prompt, some example text to give the transformer running start and context on which to build the rest of the sequence. It gets fed in to decoder, the column on the right in the image above, where it's labeled "Outputs (shifted right)". Choosing a prompt that gives interesting sequences is an art in itself, called prompt engineering. It's also a great example of humans modifying their behavior to support algorithms, rather than the other way around.

Once the decoder has a partial sequence to get started with, it takes a forward pass. The end result is a set of predicted probability distributions of words, one probability distribution for each position in the sequence. At each position, the distribution shows the predicted probabilities for each next word in the vocabulary. We don't care about predicted probabilities for each established word in the sequence. They're already established. What we really care about are the predicted probabilities for the next word after the end of the prompt. There are several ways to go about choosing what that word should be, but the most straightforward is called greedy, picking the word with the highest probability.

The new next word then gets added to the sequence, substituted in at the with the "Outputs" at the bottom of the decoder, and the process is repeated until you get tired of it.

The one piece we're not quite ready to describe in detail is yet another form of masking, ensuring that when the transformer makes predictions it only looks behind, not ahead. It's applied in the block labeled "Masked Multi-Head Attention". We'll revisit this later when we can be clearer about how it's done.

# Embeddings
As we’ve described them so far, transformers are too big. For a vocabulary size N of say 50,000, the transition matrix between all pairs of words and all potential next words would have 50,000 columns and 50,000 squared (2.5 billion) rows, totaling over 100 trillion elements. That is still a stretch, even for modern hardware.

It’s not just the size of the matrices that’s the problem. In order to build a stable transition language model, we would have to provide training data illustrating every potential sequence several times at least. That would far exceed the capacity of even the most ambitious training data sets.

Fortunately, there is a workaround for both of these problems, embeddings.

In a one-hot representation of a language, there is one vector element for each word. For a vocabulary of size N that vector is an N-dimensional space. Each word represents a point in that space, one unit away from the origin along one of the many axes. I haven't figured out a great way to draw a high dimensional space, but there's a crude representation of it below.

![](https://e2eml.school/images/transformers/one_hot_vs_embedding.png)

In an embedding, those word points are all taken and rearranged (projected, in linear algebra terminology) into a lower-dimensional space. The picture above shows what they might look like in a 2-dimensional space for example. Now, instead of needing N numbers to specify a word, we only need 2. These are the (x, y) coordinates of each point in the new space. Here's what a 2-dimensional embedding might look like for our toy example, together with the coordinates of a few of the words.

![](https://e2eml.school/images/transformers/embedded_words.png)

A good embedding groups words with similar meanings together. A model that works with an embedding learns patterns in the embedded space. That means that whatever it learns to do with one word automatically gets applied to all the words right next to it. This has the added benefit of reducing the amount of training data needed. Each example gives a little bit of learning that gets applied across a whole neighborhood of words.

In this illustration I tried to show that by putting important components in one area (battery, log, program), prepositions in another (down, out), and verbs near the center (check, find, ran). In an actual embedding the groupings may not be so clear or intuitive, but the underlying concept is the same. Distance is small between words that behave similarly.

An embedding reduces the number of parameters needed by a tremendous amount. However, the fewer the dimensions in the embedded space, the more information about the original words gets discarded. The richness of a language still requires quite a bit of space to lay out all the important concepts so that they don't step on each other's toes. By choosing the size of the embedded space, we get to trade off computational load for model accuracy.

It will probably not surprise you to learn that projecting words from their one-hot representation to an embedded space involves a matrix multiplication. Projection is what matrices do best. Starting with a one-hot matrix that has one row and N columns, and moving to an embedded space of two dimensions, the projection matrix will have N rows and two columns, as shown here.

![](https://e2eml.school/images/transformers/embedding_projection.png)

This example shows how a one-hot vector, representing for example battery, pulls out the row associated with it, which contains the coordinates of the word in the embedded space. In order to make the relationship clearer, the zeros in the one-hot vector are hidden, as are all the other rows that don't get pulled out of the projection matrix. The full projection matrix is dense, each row containing the coordinates of the word it's associated with.

Projection matrices can convert the original collection of one-hot vocabulary vectors into any configuration in a space of whatever dimensionality you want. The biggest trick is finding a useful projection, one that has similar words grouped together, and one that has enough dimensions to spread them out. There are some decent pre-computed embeddings for common langauges, like English. Also, like everything else in the transformer, it can be learned during training.

In the Figure 1 architecture diagram of the original paper, here's where the embedding happens.

![](https://e2eml.school/images/transformers/architecture_embedding.png)

# Positional encoding
Up to this point, we've assumed that the positions of words are ignored, at least for any words coming before the very most recent word. Now we get to fix that using positional embeddings.

There are several ways that position information could be introduced into our embedded represetation of words, but the way it was done in the original transformer was to add a circular wiggle.

![Positional encoding introduces a circular wiggle](https://e2eml.school/images/transformers/positional_encoding.png)

The position of the word in the embedding space acts as the center of a circle. A perturbation is added to it, depending on where it falls in the order of the sequence of words. For each position, the word is moved the same distance but at a different angle, resulting in a circular pattern as you move through the sequence. Words that are close to each other in the sequence have similar perturbations, but words that are far apart are perturbed in different directions.

Since a circle is a two dimensional figure, representing a circular wiggle requires modifying two dimensions of the embedding space. If the embedding space consists of more than two dimensions (which it almost always does), the circular wiggle is repeated in all the other pairs of dimensions, but with different angular frequency, that is, it sweeps out a different number of rotations in each case. In some dimension pairs, the wiggle will sweep out many rotations of the circle. In other pairs, it will only sweep out a small fraction of a rotation. The combination of all these circular wiggles of different frequencies gives a good representation of the absolute position of a word within the sequence.

I'm still developing my intuition for why this works. It seems to add position information into the mix in a way that doesn't disrupt the learned relationships between words and attention. For a deeper dive into the math and implications, I recommend Amirhossein Kazemnejad's positional encoding tutorial.

In the canonical architecture diagram these blocks show the generation of the position code and its addition to the embedded words.

![Transformer architecture showing positional encoding](https://e2eml.school/images/transformers/architecture_positional.png)

## Positional encoding (additional details)
[Source](https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/)